In [1]:
import time 
start = time.time()

### Загрузка библиотек

In [2]:
import sys
sys.path.append('C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/!workspace/dict_ref_moduls/moduls')

import pandas as pd
import math
import os
import xlrd
import openpyxl
from dataremake import cleaning_from_first_rows
from dataremake import comments_def
from dataremake import make_column_def
from dataremake import okato_def
from dataremake import correct_name_def
from dataremake import nice_look_def
from numpy import nan
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

### Объявление функций

In [3]:
migration_direction_dict = {
                  'внутрирег.':'Миграция в пределах России внутрирегиональная',
                  'межрег.':'Миграция в пределах России межрегиональная',
                  'междунар.':'Миграция международная',
                  'мигр_СНГ':'Миграция международная со странами СНГ',
                  'зарубежн.':'Миграция международная с другими зарубежными странами',
                  'пределы Рос.':'Миграция в пределах России',
                  'внеш.':'Внешняя для региона миграция',
                  'вся_мигр.':'Вся миграция',
}

In [4]:
indicator_dict = {
            'приб.':'число прибывших',
            'выбыв.':'число выбывших',
            'мигр_пр.':'миграционный прирост',
            'мигр':'миграционный прирост'
           }

In [5]:
sex_dict = {
            'Всего мужчин и женщин':'мужчины и женщины',
            'Всего мужчин':'мужчины',
            'Всего женщин':'женщины',
            'Все':'мужчины и женщины',
            'муж':'мужчины',
            'жен':'женщины',
            'м+ж':'мужчины и женщины',
            'м':'мужчины',
            'ж':'женщины',
           }

In [6]:
country_dict = {
            'Миграция со странами дальнего зарубежья':'Зарубежные страны',
            'Абхазия':'Абхазия',
            'Австра-лия':'Австралия',
            'Афгани-стан':'Афганистан',
            'Болга-рия':'Болгария',
            'Велико-британия':'Великобритания',
            'Вьетнам':'Вьетнам',
            'Германия':'Германия',
            'Греция':'Греция',
            'Грузия':'Грузия',
            'Израиль':'Израиль',
            'Индия':'Индия',
            'Иорда-ния':'Иордания',
            'Иран':'Иран',
            'Италия':'Италия',
            'Канада':'Канада',
            'Китай':'Китай',
            'КНДР':'КНДР',
            'Латвия':'Латвия',
            'Ливан':'Ливан',
            'Ливия':'Ливия',
            'Литва':'Литва',
            'Норве-гия':'Норвегия',
            'Польша':'Польша',
            'Сербия':'Сербия',
            'Сирия':'Сирия',
            'США':'США',
            'Турция':'Турция',
            'Финлян-дия':'Финляндия',
            'Франция':'Франция',
            'Швеция ':'Швеция',
            'Эстония':'Эстония',
            'Южная Осетия':'Южная Осетия',
            'Япония':'Япония',
            'Прочие':'Другие страны',
            'Миграция со странами СНГ':'Страны СНГ',
            'Азербайджан':'Азербайджан',
            'Армения':'Армения',
            'Беларусь':'Беларусь',
            'Казахстан':'Казахстан',
            'Киргизия':'Киргизия',
            'Республика Молдова':'Республика Молдова',
            'Таджикистан':'Таджикистан',
            'Туркмения':'Туркмения',
            'Узбекистан':'Узбекистан',
            'Украина':'Украина'
           }

In [7]:
area_type_dict = {'г+с':'городское и сельское население', 
                  'г':'городское население', 
                  'с':'сельское население'}

In [8]:
age_group_dict = {"Всего в том числе в возрасте лет:":"все возраста",
                  "Всего":"все возраста",
                  "всего":"все возраста",
                  "не указано":"не указан",
                  "60 и старше":"60 лет и более",                  
                  "0":"0 лет", 
                  "0-15":"моложе трудоспособного возраста", 
                  "0-4":"0–4 года", 
                  "1":"1 год", 
                  "10":"10 лет", 
                  "10-14":"10–14 лет", 
                  "100 и старше":"100 лет и более",
                  "11":"11 лет",
                  "12":"12 лет",
                  "13":"13 лет",
                  "14":"14 лет",
                  "14 лет и более":"14 лет и более",
                  "14-29":"14–29 лет",
                  "15":"15 лет",
                  "15-19":"15–19 лет",
                  "16":"16 лет",
                  "17":"17 лет",
                  "18":"18 лет",
                  "18 лет и более":"18 лет и более",
                  "19":"19 лет",
                  "2":"2 года",
                  "20":"20 лет",
                  "20-24":"20–24 года",
                  "21":"21 год",
                  "22":"22 года",
                  "23":"23 года",
                  "24":"24 года",
                  "25":"25 лет",
                  "25-29":"25–29 лет",
                  "26":"26 лет",
                  "27":"27 лет",
                  "28":"28 лет",
                  "29":"29 лет",
                  "3":"3 года",
                  "30":"30 лет",
                  "30-34":"30–34 года",
                  "31":"31 год",
                  "32":"32 года",
                  "33":"33 года",
                  "34":"34 года",
                  "35":"35 лет",
                  "35-39":"35–39 лет",
                  "36":"36 лет",
                  "37":"37 лет",
                  "38":"38 лет",
                  "39":"39 лет",
                  "4":"4 года",
                  "40":"40 лет",
                  "40-44":"40–44 года",
                  "41":"41 год",
                  "42":"42 года",
                  "43":"43 года",
                  "44":"44 года",
                  "45":"45 лет",
                  "45-49":"45–49 лет",
                  "46":"46 лет",
                  "47":"47 лет",
                  "48":"48 лет",
                  "49":"49 лет",
                  "5":"5 лет",
                  "5-9":"5–9 лет",
                  "50":"50 лет",
                  "50-54":"50–54 года",
                  "51":"51 год",
                  "52":"52 года",
                  "53":"53 года",
                  "54":"54 года",
                  "55":"55 лет",
                  "55-59":"55–59 лет",
                  "56":"56 лет",
                  "57":"57 лет",
                  "58":"58 лет",
                  "59":"59 лет",
                  "6":"6 лет",
                  "60":"60 лет",
                  "60-64":"60–64 года",
                  "61":"61 год",
                  "62":"62 года",
                  "63":"63 года",
                  "64":"64 года",
                  "65":"65 лет",
                  "65-69":"65–69 лет",
                  "66":"66 лет",
                  "67":"67 лет",
                  "68":"68 лет",
                  "69":"69 лет",
                  "7":"7 лет",
                  "70":"70 лет",
                  "70-74":"70–74 года",
                  "71":"71 год",
                  "72":"72 года",
                  "73":"73 года",
                  "74":"74 года",
                  "75":"75 лет",
                  "75-79":"75–79 лет",
                  "76":"76 лет",
                  "77":"77 лет",
                  "78":"78 лет",
                  "79":"79 лет",
                  "8":"8 лет",
                  "80":"80 лет",
                  "80-84":"80–84 года",
                  "81":"81 год",
                  "82":"82 года",
                  "83":"83 года",
                  "84":"84 года",
                  "85":"85 лет",
                  "85-89":"85–89 лет",
                  "86":"86 лет",
                  "87":"87 лет",
                  "88":"88 лет",
                  "89":"89 лет",
                  "9":"9 лет",
                  "90":"90 лет",
                  "90-94":"90–94 года",
                  "91":"91 год",
                  "92":"92 года",
                  "93":"93 года",
                  "94":"94 года",
                  "95":"95 лет",
                  "95-99":"95–99 лет",
                  "96":"96 лет",
                  "97":"97 лет",
                  "98":"98 лет",
                  "99":"99 лет",
                  "старше трудоспособного":"старше трудоспособного возраста",
                  "трудоспособном":"в трудоспособном возрасте",
                  '15-17':'15-17 лет', 
                  '18-19':'18-19 лет',
                  'в т.ч.            0-17 лет.':'0–17 лет',
                  ' до 1 года':'до 1 года',
                  'в т.ч. в трудоспос. возр.':'в трудоспособном возрасте',
                  ' Всего  ':'все возраста',
                  ' 1-4 года ':'1–4 года'
                 }

In [9]:
age_bride_groom_dict = {
                  "18-19":"18–19 лет",
                  "55-59":"55–59 лет",
                  "35-39":"35–39 лет",
                  "60 лет и более":"60 лет и более",
                  "60 и старше":"60 лет и более",
                  "возраст неизве-стен":"не указан",
                  "25-29":"25–29 лет",
                  "20-24":"20–24 года",
                  "30-34":"30–34 года",
                  "до 16":"до 16 лет",
                  "40-44":"40–44 года",
                  "не указано":"не указан",
                  "45-49":"45–49 лет",
                  "всего":"все возраста",
                  "до 16 лет":"до 16 лет",
                  "16-17":"16–17 лет",
                  "50-54":"50–54 года"}

In [10]:
deat_cause_dict = {
'256-258,307 (ДТП)':'ДТП',
'число умерших без внешних причин смерти':'без внешних причин смерти',
'Болезни глаза и его придаточного аппарата':'болезни глаза и его придаточного аппарата',
'192-193  (Болезни кожи и подкожной клетчатки)':'болезни кожи и подкожной клетчатки',
'194-197  (Болезни костно-мышечной системы и соединительной ткани)':'болезни костно-мышечной системы и соединительной ткани',
'88-90  (Болезни крови, кроветворных органов и отдельные нарушения с вовлечением иммунного механизма)':'болезни крови, кроветворных органов и отдельные нарушения с вовлечением иммунного механизма',
'198-206 (Болезни мочеполовой системы)':'болезни мочеполовой системы',
'106-117,330  (Болезни нервной системы)':'болезни нервной системы',
'156-172  (Болезни органов дыхания)':'болезни органов дыхания',
'173-191,315 (Болезни органов пищеварения)':'болезни органов пищеварения',
'182-184  (Болезни печени)':'болезни печени',
'187-188,315 (Болезни поджелудочной железы)':'болезни поджелудочной железы',
'121-155  (Болезни системы кровообращения)':'болезни системы кровообращения',
'119-120  (Болезни уха и сосцевидного отростка)':'болезни уха и сосцевидного отростка',
'91-95  (Болезни эндокринной системы, расстройства питания и нарушения обмена веществ)':'болезни эндокринной системы, расстройства питания и нарушения обмена веществ',
'Болезнь, вызванная вирусом иммунодефицита человека (ВИЧ)':'болезнь, вызванная вирусом иммунодефицита человека (ВИЧ)',
'37-42(Вирусные гепатиты)':'вирусные гепатиты',
'256-305,307  (Внешние причины смертности)':'внешние причины смерти',
'232-241  (Врожденные аномалии (пороки разви-тия), деформации и хромосомные нарушения)':'врожденные аномалии (пороки развития), деформации и хромосомные нарушения',
'Всего умерших (1-245,256-307,315-318,320,325)':'все причины',
'54-86 (Злокачественные образования)':'злокачественные новообразования',
'127-128  (Инфаркт миокарда)':'инфаркт миокарда',
'127-133  (Ишемическая болезнь сердца)':'ишемическая болезнь сердца',
'207-219 (Материнская смертность без кода 17)':'материнская смертность',
'Менингит, за исключением менингита при инфекционных и паразитарных заболеваниях':'менингит, за исключением менингита при инфекционных и паразитарных заболеваниях',
'1-53,325(Некоторые инфекционные и паразитарные болезни)':'некоторые инфекционные и паразитарные болезни',
'54-87(Новообразования)':'новообразования',
'17,207-220 (Осложнения беременности, родов и послеродового периода)':'осложнения беременности, родов и послеродового периода',
'141-144 острое нарушение мозгового кровообращения':'острое нарушение мозгового кровообращения',
'221-231  (Отдельные состояния, возникающие в перинатальном периоде)':'отдельные состояния, возникающие в перинатальном периоде',
'159-162  (Пневмония)':'пневмония',
'286-298  (повреждения с неопределенными намерениями)':'повреждения с неопределенными намерениями',
'96-105 (Психические расстройства и расстройства поведения)':'психические расстройства и расстройства поведения',
'Рак in situ, доброкачественные неопределенного и неизвестного характера новообразования':'рак in situ, доброкачественные новообразования неопределенного и неизвестного характера',
'283-284  (Самоубийства)':'самоубийства',
'91-93  (Сахарный диабет)':'сахарный диабет',
'242-245  (Симптомы, признаки, отклонения от нормы, выявленные при клинических и лабораторных исследованиях, не классифицированные в других рубриках)':'симптомы, признаки, отклонения от нормы, выявленные при клинических и лабораторных исследованиях, не классифицированные в других рубриках',
'Случайное отравление (воздействие) алкоголем':'случайное отравление (воздействие) алкоголем',
'Старость':'старость',
'Всех видов транспортные несчастные случаи (256-268, 296, 307)':'транспортные несчастные случаи всех видов',
'9-11(Туберкулез всех форм)':'туберкулез всех форм',
'Убийство (нападение, насилие)':'убийство (нападение, насилие)',
'141-150  (Цереброваскулярные болезни)':'цереброваскулярные болезни',
'173-175  (Язва)':'язва'
}

In [11]:
def correct_migration_direction(row):
    r = migration_direction_dict[row['migration_direction']]
    return r

def correct_indicator(row):
    r = indicator_dict[row['bad_indicator_name']]
    return r

def correct_sex(row):
    r = sex_dict[row['sex']]
    return r

def correct_country(row):
    r = country_dict[row['country']]
    return r

def correct_area_type(row):
    r = area_type_dict[row['area_type']]
    return r

def correct_age_group(row):
    try:
        r = age_group_dict[str(row['age_group'])]
        return r
    except KeyError:
        return row['age_group']

def correct_bride_age(row):
    r = age_bride_groom_dict[row['bride_age']]
    return r

def correct_groom_age(row):
    r = age_bride_groom_dict[row['groom_age']]
    return r

def correct_death(row):
    try:
        r = deat_cause_dict[row['death_cause']]
        return r
    except KeyError:
        return row['death_cause']


In [12]:
def make_good_def(data_inner, 
                 year_inner,
                 original_unit_inner,
                 indicator_name_inner=None
                 ):
    data_inner.columns = list(range(len(data_inner.columns)))

    data_inner = data_inner.drop(index = [0, 1, 2, 3, 5, 6]).reset_index(drop = True)

    
    data_inner.at[0,0] = 'age_group'    
    data_inner.at[0,1] = 'year_of_birth'
    data_inner.at[0,2] = 'всего'


    data_inner.columns = data_inner.loc[0].tolist()

    data_inner = data_inner.drop(index = [0]).reset_index(drop = True)    
    


    data_inner = data_inner.melt(id_vars=['age_group', 'year_of_birth'])
    data_inner.columns = ['age_group', 'year_of_birth', 'birth_order', 'indicator_value']  
    data_inner.age_group = data_inner.apply(correct_age_group, axis=1)


    data_inner['year'] = year_inner    
    data_inner['original_unit'] = original_unit_inner
    data_inner['indicator_unit'] = data_inner['original_unit']

    data_inner['object_okato'] = data_inner.apply(okato_def, axis = 1)   


    data_inner['indicator_name'] = indicator_name_inner


    return data_inner

In [13]:
list_1 = list(deat_cause_dict.values())
list_2 = list(age_group_dict.values())

def make_good_def_2(data_inner, 
                    year_inner,
                    original_unit_inner,                    
                    object_name_inner,
                    area_type_inner,
                    sex_inner,
                    indicator_name_inner = None
                    ):
    
    data_inner.columns = list(range(len(data_inner.columns)))

    data_inner = data_inner.drop(index = [0, 1, 2, 4, 5, 6]).reset_index(drop = True)
    data_inner = data_inner.drop(columns=[0])
    data_inner.columns = list(range(len(data_inner.columns)))

    
    data_inner.at[0,0] = 'death_cause'    



    data_inner.columns = data_inner.loc[0].tolist()

    data_inner = data_inner.drop(index = [0]).reset_index(drop = True)    
    


    data_inner = data_inner.melt(id_vars=['death_cause'])
    data_inner.columns = ['death_cause', 'age_group', 'indicator_value']  
    # data_inner.age_group = data_inner.apply(correct_age_group, axis=1)


    data_inner['year'] = year_inner    
    data_inner['original_unit'] = original_unit_inner
    data_inner['indicator_unit'] = data_inner['original_unit']
    data_inner['object_name'] = object_name_inner
    data_inner['object_okato'] = data_inner.apply(okato_def, axis = 1)   
    data_inner['area_type'] = area_type_inner
    data_inner['sex'] = sex_inner
    data_inner['death_cause'] = data_inner.apply(correct_death, axis = 1)   
    data_inner['age_group'] = data_inner.apply(correct_age_group, axis = 1)   

    data_inner = data_inner[(data_inner.death_cause.isin(list_1)) & (data_inner.age_group.isin(list_2))]


    # data_inner['indicator_name'] = indicator_name_inner



    return data_inner

### Объявление констант

In [14]:
incoming_data_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/1_inсoming_data/'
data_name = '81_101'
directory_name = '31_39'


### Загрузка датасетов

In [15]:
data_work = pd.read_excel(incoming_data_path + data_name + '.xlsx')
    

In [16]:
data_work.head(10)

,"Распределение умерших по полу, возрастным группам и причинам смерти\n в 2020 году",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,Российская Федерация городское население,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,оба пола,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Шифр МКБ-Х,Причина смерти,до 1 года,0-27 дней,1 год,2 года,3 года,4 года,1-4 года,5-9 лет,10-14 лет,15-19 лет,20-24 года,25-29 лет,30-34 лет,35-39 лет,40-44 лет,45-49 лет,50-54 лет,55-59 лет,60-64 лет,65-69 лет,70-74 лет,75-79 лет,80-84 лет,85 и более лет,70 и более лет,не указан,Всего,в т.ч. в трудоспос. возр.,в т.ч. 0-17 лет.,в т.ч. 15-17 лет.,по врач. свид.пат. анат.
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,А,Б,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
7,1,Холера,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2,Брюшной тиф,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,3,Паратифы,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
data_work = make_good_def(data_inner = data_work, 
                          year_inner = 2020,
                          original_unit_inner = 'человек',
                          indicator_name_inner = 'Число родившихся живыми',
                 )

In [29]:
start_circle = time.time()
xl = pd.ExcelFile(incoming_data_path + data_name + '.xlsx')
index = [0]
data_work = pd.DataFrame(index=index, columns = ['indicator_name', 'indicator_value'])
i = 1
for sh_name in xl.sheet_names:
    data_work_c = pd.read_excel(incoming_data_path + data_name + '.xlsx', sheet_name = sh_name)
    area_type = area_type_dict[sh_name.split('_')[1].lower()]
    sex = sex_dict[sh_name.split('_')[2]]
    object_name = data_work_c.loc[0,data_work_c.columns[0]].replace(area_type, '').strip()

    data_work_c = make_good_def_2(data_inner = data_work_c, 
                                  year_inner = 2020,
                                  original_unit_inner = 'человек',
    #                               indicator_name_inner = 'Число родившихся живыми по возрасту матери и очередности рождения',
                                  object_name_inner = object_name,
                                  area_type_inner = area_type,
                                  sex_inner = sex)
    data_work = pd.concat([data_work, data_work_c], ignore_index=True, sort=False)
    print("{:.3%}".format((i/864)), 'time {:.0f}'.format(time.time() - start_circle),end='\r')
    i += 1

data_work = data_work.dropna(subset=['death_cause']).reset_index(drop=True)


xl.close()


In [72]:
xl = pd.ExcelFile(incoming_data_path + data_name + '.xlsx')
print(len(list(xl.sheet_names)))
xl.close()

864


In [66]:
for u in sorted(list(data_work.death_cause.unique())):
    print(u)

for u in sorted(list(data_work.age_group.unique())):
    print(u)

ДТП
без внешних причин смерти
болезни глаза и его придаточного аппарата
болезни кожи и подкожной клетчатки
болезни костно-мышечной системы и соединительной ткани
болезни крови, кроветворных органов и отдельные нарушения с вовлечением иммунного механизма
болезни мочеполовой системы
болезни нервной системы
болезни органов дыхания
болезни органов пищеварения
болезни печени
болезни поджелудочной железы
болезни системы кровообращения
болезни уха и сосцевидного отростка
болезни эндокринной системы, расстройства питания и нарушения обмена веществ
болезнь, вызванная вирусом иммунодефицита человека (ВИЧ)
вирусные гепатиты
внешние причины смерти
врожденные аномалии (пороки развития), деформации и хромосомные нарушения
все причины
злокачественные новообразования
инфаркт миокарда
ишемическая болезнь сердца
материнская смертность
менингит, за исключением менингита при инфекционных и паразитарных заболеваниях
некоторые инфекционные и паразитарные болезни
новообразования
осложнения беременности, родо

In [3]:
prepared_data_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/2_prepared_data/'
data_for_concat = 'data_81_101.csv'
data_work = pd.read_csv(prepared_data_path + data_for_concat, index_col = 'Unnamed: 0')

In [4]:
data_work.head(30)

,indicator_name,indicator_value,death_cause,age_group,year,original_unit,indicator_unit,object_name,object_okato,area_type,sex
0,NaN,3,туберкулез всех форм,до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
1,NaN,1,вирусные гепатиты,до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
2,NaN,5,"болезнь, вызванная вирусом иммунодефицита чело...",до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
3,NaN,140,некоторые инфекционные и паразитарные болезни,до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
4,NaN,36,злокачественные новообразования,до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
5,NaN,10,"рак in situ, доброкачественные новообразования...",до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
6,NaN,46,новообразования,до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
7,NaN,15,"болезни крови, кроветворных органов и отдельны...",до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
8,NaN,1,сахарный диабет,до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины
9,NaN,32,"болезни эндокринной системы, расстройства пита...",до 1 года,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины


### Работа с датасетами

In [5]:
data_work.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190110 entries, 0 to 190109
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   indicator_name   0 non-null       float64
 1   indicator_value  190110 non-null  int64  
 2   death_cause      190110 non-null  object 
 3   age_group        190110 non-null  object 
 4   year             190110 non-null  float64
 5   original_unit    190110 non-null  object 
 6   indicator_unit   190110 non-null  object 
 7   object_name      190110 non-null  object 
 8   object_okato     186150 non-null  float64
 9   area_type        190110 non-null  object 
 10  sex              190110 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 17.4+ MB


In [7]:
for col in data_work.columns:
    print(col)
    print(data_work[col].unique())

indicator_name
[nan]
indicator_value
[   3    1    5 ... 6693 3641 3052]
death_cause
['туберкулез всех форм' 'вирусные гепатиты'
 'болезнь, вызванная вирусом иммунодефицита человека (ВИЧ)'
 'некоторые инфекционные и паразитарные болезни'
 'злокачественные новообразования'
 'рак in situ, доброкачественные новообразования неопределенного и неизвестного характера'
 'новообразования'
 'болезни крови, кроветворных органов и отдельные нарушения с вовлечением иммунного механизма'
 'сахарный диабет'
 'болезни эндокринной системы, расстройства питания и нарушения обмена веществ'
 'психические расстройства и расстройства поведения'
 'менингит, за исключением менингита при инфекционных и паразитарных заболеваниях'
 'болезни нервной системы' 'болезни глаза и его придаточного аппарата'
 'болезни уха и сосцевидного отростка' 'инфаркт миокарда'
 'ишемическая болезнь сердца' 'острое нарушение мозгового кровообращения'
 'цереброваскулярные болезни' 'болезни системы кровообращения' 'пневмония'
 'болезни

In [8]:
data_101 = data_work[data_work.age_group.isin(['0–17 лет', 'до 1 года', 'в трудоспособном возрасте', 'все возраста'])]

In [12]:
data_101.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152088 entries, 0 to 190109
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   indicator_name   152088 non-null  object 
 1   indicator_value  152088 non-null  int64  
 2   death_cause      152088 non-null  object 
 3   age_group        152088 non-null  object 
 4   year             152088 non-null  float64
 5   original_unit    152088 non-null  object 
 6   indicator_unit   152088 non-null  object 
 7   object_name      152088 non-null  object 
 8   object_okato     148920 non-null  float64
 9   area_type        152088 non-null  object 
 10  sex              152088 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 13.9+ MB


In [83]:
data_81 = data_work[data_work.age_group.isin([nan, '0–5 лет', '5 лет', 'до 1 года', '1–4 года'])]

In [84]:
data_81.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76044 entries, 0 to 189977
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   indicator_name   0 non-null      float64
 1   indicator_value  76044 non-null  int64  
 2   death_cause      76044 non-null  object 
 3   age_group        76044 non-null  object 
 4   year             76044 non-null  float64
 5   original_unit    76044 non-null  object 
 6   indicator_unit   76044 non-null  object 
 7   object_name      76044 non-null  object 
 8   object_okato     74460 non-null  float64
 9   area_type        76044 non-null  object 
 10  sex              76044 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 7.0+ MB


In [86]:
print(len(data_81.death_cause.unique())*
      len(data_81.object_name.unique())*
      len(data_81.area_type.unique())*
      len(data_81.sex.unique()))

38016


In [85]:
data_81.indicator_name = 'Число детей, умерших в возрасте до 5 лет по причинам смерти'

In [94]:
data_81.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113268 entries, 0 to 113267
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   indicator_name   113268 non-null  object 
 1   indicator_value  113268 non-null  int64  
 2   death_cause      113268 non-null  object 
 3   age_group        113268 non-null  object 
 4   year             113268 non-null  float64
 5   original_unit    76044 non-null   object 
 6   indicator_unit   76044 non-null   object 
 7   object_name      113268 non-null  object 
 8   object_okato     111684 non-null  float64
 9   area_type        113268 non-null  object 
 10  sex              113268 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 9.5+ MB


In [93]:
data_81 = pd.concat([data_81, test], ignore_index=True, sort=False)

In [81]:
data_81

Index(['indicator_name', 'indicator_value', 'death_cause', 'age_group', 'year',
       'original_unit', 'indicator_unit', 'object_name', 'object_okato',
       'area_type', 'sex'],
      dtype='object')

In [88]:
test = data_81.groupby(['indicator_name', 'death_cause', 'year',
       'object_name', 'object_okato', 'area_type', 'sex'], as_index=False).sum()

In [91]:
test['age_group'] = '0–5 лет'

In [92]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37224 entries, 0 to 37223
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   indicator_name   37224 non-null  object 
 1   death_cause      37224 non-null  object 
 2   year             37224 non-null  float64
 3   object_name      37224 non-null  object 
 4   object_okato     37224 non-null  float64
 5   area_type        37224 non-null  object 
 6   sex              37224 non-null  object 
 7   indicator_value  37224 non-null  int64  
 8   age_group        37224 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 2.6+ MB


In [68]:
i = 1
for d in data_81.death_cause.unique()[:1]:
    for o in data_81.object_name.unique()[:1]:
        for a in data_81.area_type.unique()[:1]:
            for s in data_81.sex.unique()[:1]:
                
                x = data_81[(data_81.death_cause == d) & 
                                       (data_81.object_name == o) & 
                                       (data_81.area_type == a) & 
                                       (data_81.sex == s)]
                summa = data_81[(data_81.death_cause == d) & 
                                       (data_81.object_name == o) & 
                                       (data_81.area_type == a) & 
                                       (data_81.sex == s)]['indicator_value'].sum()                
                x.indicator_value = summa
                x.age_group = '0–5 лет'
                y = x.drop_duplicates()
                display(y)
                # data_81 = pd.concat([data_81, y], ignore_index=True, sort=False)
                print(i,end='\r')
                i += 1
  


,indicator_name,indicator_value,death_cause,age_group,year,original_unit,indicator_unit,object_name,object_okato,area_type,sex
0,"Число детей, умерших в возрасте до 5 лет по пр...",4,туберкулез всех форм,0–5 лет,2020.0,человек,человек,Российская Федерация,643.0,городское население,мужчины и женщины


In [95]:
for col in data_81.columns:
    print(col)
    print(data_81[col].unique())

indicator_name
['Число детей, умерших в возрасте до 5 лет по причинам смерти']
indicator_value
[   3    1    5  140   36   10   46   15   32    0    6  168    8    9
   51   92  151    4    2   21 2498 1005  201   13   14   56  225 4573
 4348   72  138  143   24  183   34   41   75   20    7  152   52   95
  380 1148   62  768   89   19   22   26   49 1441  559  111   40  134
 2615 2481   38   69   70  104   18   79   60  242  650  408   17   12
   73   25   43   59   11 1057  446   90   16   91 1958 1867   37   35
  498  360   65   86  124  910  127  131 1916 1785   28   50  106   29
   44  232  611  379   66  523  235   63   80 1103 1023   64   30  135
  333  198   45   58  387  145  813  762   42   27   97  278  181  231
   48  233   76  178  275   39 3408 1385  328   88  356 6489   23 6133
  100  188  195  289   33  204   84  139  612 1759 1147  158 1964  794
  174  214 3718 3504   93   94  377  983  606   98  117 1444  591  154
  142 2771 2629  101  121  776  541  588  319 1153 11

In [96]:
data_work = pd.concat([data_81, data_101], ignore_index=True, sort=False)

In [98]:
for col in data_work.columns:
    print(col)
    print(data_work[col].unique())

indicator_name
['Число детей, умерших в возрасте до 5 лет по причинам смерти'
 'Число умерших по причинам смерти']
indicator_value
[   3    1    5 ... 6693 3641 3052]
death_cause
['туберкулез всех форм' 'вирусные гепатиты'
 'болезнь, вызванная вирусом иммунодефицита человека (ВИЧ)'
 'некоторые инфекционные и паразитарные болезни'
 'злокачественные новообразования'
 'рак in situ, доброкачественные новообразования неопределенного и неизвестного характера'
 'новообразования'
 'болезни крови, кроветворных органов и отдельные нарушения с вовлечением иммунного механизма'
 'сахарный диабет'
 'болезни эндокринной системы, расстройства питания и нарушения обмена веществ'
 'психические расстройства и расстройства поведения'
 'менингит, за исключением менингита при инфекционных и паразитарных заболеваниях'
 'болезни нервной системы' 'болезни глаза и его придаточного аппарата'
 'болезни уха и сосцевидного отростка' 'инфаркт миокарда'
 'ишемическая болезнь сердца' 'острое нарушение мозгового кровоо

In [99]:
prepared_data_path = 'C:/Users/User/Desktop/Росстат/Проект_Датасеты/Датасеты/Демография/!Датасет_демография/2_prepared_data/'

In [100]:
data_work.to_csv(prepared_data_path + 'data_81_101'+'.csv')

In [ ]:
end = time.time()
duration = end-start
print(int(duration), 'сек. или', int(duration//60), 'мин. и ', int(duration % 60), 'сек.')

1625 сек. или 27 мин. и  5 сек.
